# GAによる関数最大値探索 y 

### 1.必要モジュールのインポート

In [1]:
import random
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

### 2.GAのシミュレーションクラスと個体（染色体）クラスの生成

In [2]:
class SimpleGA:
    def __init__(self):
        self.N = 15 # 個体数
        self.ITERATION = 100 # 世代数
        self._initialize()

    def _initialize(self):
        self.pool = [Chromosome() for i in range(self.N)]

    def _mutateAll(self):
        for i in range(self.N):
            self.pool[i].mutate()

    def _tournamentSelection(self):
        pool_next = []
        while len(pool_next) < self.N:
            offspring1 = copy.deepcopy(self.pool[random.randrange(self.N)])
            offspring2 = copy.deepcopy(self.pool[random.randrange(self.N)])
            if offspring1.getFittness() > offspring2.getFittness():
                pool_next.append(offspring1)
            else:
                pool_next.append(offspring2)
        self.pool = pool_next[:]

    def _printStatus(self, iteration):
        print("generation\t" + str(iteration))
        for c in self.pool:
            print("\t" + str(c))

    def evolve(self):
        for i in range(self.ITERATION):
            self._printStatus(i)
            self._tournamentSelection()
            self._mutateAll()

In [3]:
class Chromosome:
    def __init__(self):
        self.LENGTH = 8 # 遺伝子数
        self.MUTATION_RATE = 0.05 # 突然変異率
        self.gene = [random.randint(0,1) for i in range(self.LENGTH)]

    def mutate(self):
        for i in range(self.LENGTH):
            if random.random() < self.MUTATION_RATE:
                self.gene[i] = 1 - self.gene[i]
    
    def getVal(self): # 遺伝子型から表現型への発現
        value = 0.
        for g in self.gene:
            value *= 2
            value += g
        value = value / (2 ** self.LENGTH - 1.0)
        return value
    
    def getFittness(self): # 適応度評価
        result = self.getVal()
        return - (result - 1/2) * (result - 1/2) + 1/4 

    def __str__(self):
        result = ""
        for g in self.gene:
            result += str(g)
        result += "\t" + str(self.getFittness())
        return result

In [4]:
ga = SimpleGA()
ga.evolve()

generation	0
	11000011	0.1799307958477509
	11001011	0.1623375624759708
	10100100	0.22951172625913108
	01101010	0.24289119569396386
	01000100	0.19555555555555554
	00100111	0.12955017301038063
	01000100	0.19555555555555554
	00110110	0.16692041522491352
	11101111	0.058808150711264884
	01000011	0.19371011149557862
	11110110	0.03404844290657438
	01000010	0.1918339100346021
	10011101	0.23661668589004228
	01001100	0.20921184159938486
	11010111	0.1322568242983468
generation	1
	00100111	0.12955017301038063
	01000011	0.19371011149557862
	11010111	0.1322568242983468
	10011101	0.23661668589004228
	11001100	0.15999999999999998
	11000001	0.18402153018069975
	10011101	0.23661668589004228
	01000100	0.19555555555555554
	01101010	0.24289119569396386
	10100100	0.22951172625913108
	10110100	0.20761245674740483
	11000011	0.1799307958477509
	11000011	0.1799307958477509
	01101110	0.24529027297193387
	11010111	0.1322568242983468
generation	2
	10100100	0.22951172625913108
	11000011	0.1799307958477509
	01101110

## SimpleGAクラスを継承したアニメーション用クラスの生成

In [5]:
%matplotlib nbagg
class SimpleGA_animation(SimpleGA):
    # コンストラクタ
    def __init__(self):
        super(SimpleGA_animation, self).__init__()
        self.fig = plt.figure(figsize=(5, 5))
        self.ims = []
    
    # SimpleGAのevolveメソッドをオーバーライド
    def evolve(self):
        for i in range(self.ITERATION):
            # self._printStatus(i)
            # self._printAverage()
            self._tournamentSelection()
            self._mutateAll()
            self.snapshot(i)
        self.output_animation()
    
    # スナップショット
    def snapshot(self, t):
        val_list=[]
        im=plt.plot(np.linspace(0,1,100), [ - (n - 1/2) * (n - 1/2) + 1/4 for n in np.linspace(0,1,100)], color='black', linestyle='solid')
        plt.title("GA:遺伝的アルゴリズムGAによるシミュレーション", fontname="MS Gothic")
        im += [plt.text(0,0.22,'generation: {}'.format(t))]
        im += [plt.text(0,0.24,'N = {}'.format(self.N))]
        im += [plt.scatter([chromosome.getVal() for chromosome in self.pool], [chromosome.getFittness() for chromosome in self.pool],c='b')]
        
        self.ims.append(im)
        
    # 出力
    def output_animation(self):
        global ani # Jupyterでアニメーション表示のためにglobalが必要
        ani = animation.ArtistAnimation(self.fig, self.ims)
        # ani.save('./anim.mp4', writer="ffmpeg")
        ani.save('./anim2.gif', writer="pillow")

### アニメーション

In [6]:
ga = SimpleGA_animation()
ga.evolve()

<IPython.core.display.Javascript object>